In [ ]:
# Import necessary libraries
import numpy as np
import os
import networkx as nx
import matplotlib.pyplot as plt
from collections import deque
import random
import time

: 

In [11]:
def import_wiki_vote_data(file_path):
    # Initialize an empty list to store the edges
    edges = []

    # Open and read the file
    with open(file_path, 'r') as file:
        for line in file:
            # Skip comments and empty lines
            if line.startswith('#') or line.strip() == '':
                continue
            
            # Split the line into source and target nodes and convert them to integers
            source, target = map(int, line.strip().split())
            
            # Add the edge to the list
            edges.append([source, target])
    
    # Convert the list to a numpy array and remove duplicate edges
    nodes_connectivity_list = np.unique(np.array(edges), axis=0)
    
    return nodes_connectivity_list

In [12]:
# Update the file path to use a relative path
file_path = os.path.join("data", "Wiki-Vote.txt")
nodes_connectivity_list_wiki = import_wiki_vote_data(file_path)

In [13]:
# Print a few entries of the nodes_connectivity_list_wiki array
print("First 5 entries of nodes_connectivity_list_wiki:")
for i in range(min(5, len(nodes_connectivity_list_wiki))):
    print(nodes_connectivity_list_wiki[i])

print("\nShape of nodes_connectivity_list_wiki:", nodes_connectivity_list_wiki.shape)


First 5 entries of nodes_connectivity_list_wiki:
[ 3 28]
[ 3 30]
[ 3 39]
[ 3 54]
[  3 108]

Shape of nodes_connectivity_list_wiki: (103689, 2)


In [14]:
# Check for repeated rows (edges) in nodes_connectivity_list_wiki
unique_edges, counts = np.unique(nodes_connectivity_list_wiki, axis=0, return_counts=True)
repeated_edges = unique_edges[counts > 1]

if len(repeated_edges) > 0:
    print("Repeated edges found:")
    for edge in repeated_edges:
        print(edge)
else:
    print("No repeated edges found.")


No repeated edges found.


In [15]:
def calculate_edge_betweenness(G):

    start_time = time.time()
    edge_betweenness = {edge: 0 for edge in G.edges()}
    nodes = list(G.nodes())
    total_nodes = len(nodes)
    
    print(f"Starting edge betweenness calculation for {total_nodes} nodes and {len(G.edges())} edges")
    
    for i, source in enumerate(nodes):
        if i % 100 == 0:  # Print progress every 100 nodes
            elapsed_time = time.time() - start_time
            print(f"Processing node {i+1}/{total_nodes} ({(i+1)/total_nodes*100:.2f}%). Elapsed time: {elapsed_time:.2f} seconds")
        
        # Run BFS from the source node
        distances = {node: float('inf') for node in nodes}
        distances[source] = 0
        queue = deque([(source, 0)])
        paths = {node: [] for node in nodes}
        paths[source] = [[source]]
        
        while queue:
            node, dist = queue.popleft()
            for neighbor in G.neighbors(node):
                if distances[neighbor] > dist + 1:
                    distances[neighbor] = dist + 1
                    paths[neighbor] = [path + [neighbor] for path in paths[node]]
                    queue.append((neighbor, dist + 1))
                elif distances[neighbor] == dist + 1:
                    paths[neighbor].extend([path + [neighbor] for path in paths[node]])
        
        # Calculate edge betweenness
        for target in nodes:
            if target != source:
                total_paths = len(paths[target])
                if total_paths > 0:
                    for path in paths[target]:
                        for i in range(len(path) - 1):
                            edge = (path[i], path[i+1])
                            if edge in edge_betweenness:
                                edge_betweenness[edge] += 1 / total_paths
                            else:
                                edge_betweenness[(path[i+1], path[i])] += 1 / total_paths
    
    print("Finished calculating edge betweenness. Normalizing results...")
    
    # Normalize by the number of node pairs
    n = len(nodes)
    normalization_factor = 1 / ((n * (n - 1)) / 2)
    for edge in edge_betweenness:
        edge_betweenness[edge] *= normalization_factor
    
    total_time = time.time() - start_time
    print(f"Edge betweenness calculation completed in {total_time:.2f} seconds")
    
    return edge_betweenness

In [16]:
def Girvan_Newman_one_level(nodes_connectivity_list):
    # Create a graph from the edge list
    G = nx.Graph()
    G.add_edges_from(nodes_connectivity_list)
    
    # Calculate edge betweenness
    edge_betweenness = calculate_edge_betweenness(G)
    
    # Find the edge with the highest betweenness
    max_betweenness_edge = max(edge_betweenness, key=edge_betweenness.get)
    
    # Remove the edge with the highest betweenness
    G.remove_edge(*max_betweenness_edge)
    
    # Find connected components (communities)
    communities = list(nx.connected_components(G))
    
    # Create a mapping from original node IDs to new contiguous node IDs
    node_mapping = {node: idx for idx, node in enumerate(G.nodes())}
    
    # Create the graph partition array
    n = len(G.nodes())
    graph_partition = np.zeros(n, dtype=int)
    
    for community in communities:
        community_id = min(community)
        for node in community:
            graph_partition[node_mapping[node]] = node_mapping[community_id]
    
    return graph_partition

In [18]:
graph_partition_wiki = Girvan_Newman_one_level(nodes_connectivity_list_wiki)

Starting edge betweenness calculation for 7115 nodes and 100762 edges
Processing node 1/7115 (0.01%). Elapsed time: 0.06 seconds
Processing node 101/7115 (1.42%). Elapsed time: 98.37 seconds
Processing node 201/7115 (2.83%). Elapsed time: 189.58 seconds
Processing node 301/7115 (4.23%). Elapsed time: 276.43 seconds
Processing node 401/7115 (5.64%). Elapsed time: 366.04 seconds
Processing node 501/7115 (7.04%). Elapsed time: 453.63 seconds
Processing node 601/7115 (8.45%). Elapsed time: 538.41 seconds
Processing node 701/7115 (9.85%). Elapsed time: 619.03 seconds
Processing node 801/7115 (11.26%). Elapsed time: 697.23 seconds
Processing node 901/7115 (12.66%). Elapsed time: 779.50 seconds
Processing node 1001/7115 (14.07%). Elapsed time: 867.57 seconds
Processing node 1101/7115 (15.47%). Elapsed time: 958.11 seconds
Processing node 1201/7115 (16.88%). Elapsed time: 1058.89 seconds
Processing node 1301/7115 (18.29%). Elapsed time: 1154.93 seconds
Processing node 1401/7115 (19.69%). Elaps

In [17]:
print("Graph partition after one level of Girvan-Newman algorithm:")
print(graph_partition_wiki)

Starting edge betweenness calculation for 7115 nodes and 100762 edges
Processing node 1/7115 (0.01%). Elapsed time: 0.06 seconds


KeyboardInterrupt: 

In [5]:
def Girvan_Newman(nodes_connectivity_list):
    # Create a graph from the edge list
    G = nx.Graph()
    G.add_edges_from(nodes_connectivity_list)
    
    n = G.number_of_nodes()
    community_mat = []
    
    while G.number_of_edges() > 0:
        # Apply one level of Girvan-Newman algorithm
        graph_partition = Girvan_Newman_one_level(G.edges())
        
        # Create a mapping from original node IDs to contiguous indices
        node_to_index = {node: i for i, node in enumerate(G.nodes())}
        
        # Ensure the partition is a complete list of node assignments
        complete_partition = np.zeros(n, dtype=int)
        for i, node in enumerate(G.nodes()):
            if node_to_index[node] < len(graph_partition):
                complete_partition[i] = graph_partition[node_to_index[node]]
            else:
                # Assign a new community ID for nodes not in graph_partition
                complete_partition[i] = len(graph_partition)
        
        # Add the current partition to the community matrix
        community_mat.append(complete_partition)
        
        # Find the edge with the highest betweenness
        edge_betweenness = calculate_edge_betweenness(G)
        max_betweenness_edge = max(edge_betweenness, key=edge_betweenness.get)
        
        # Remove the edge with the highest betweenness
        G.remove_edge(*max_betweenness_edge)
    
    # Convert the list of partitions to a numpy array
    community_mat = np.array(community_mat).T
    
    return community_mat

In [ ]:
# Run Girvan-Newman algorithm on the wiki-vote dataset
community_mat_wiki = Girvan_Newman(nodes_connectivity_list_wiki)

print("Community matrix after running full Girvan-Newman algorithm:")
print(community_mat_wiki)

# Print the shape of the community matrix to understand its dimensions
print("\nShape of the community matrix:")
print(community_mat_wiki.shape)

# Print the number of unique communities at each level
print("\nNumber of unique communities at each level:")
for level in range(community_mat_wiki.shape[1]):
    unique_communities = len(np.unique(community_mat_wiki[:, level]))
    print(f"Level {level}: {unique_communities} communities")

In [43]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

def visualise_dendogram(community_mat):
    # Transpose the community matrix to have nodes as columns and levels as rows
    community_mat_T = community_mat.T

    # Create a linkage matrix
    linkage_matrix = linkage(community_mat_T, method='ward')

    # Create a new figure with a larger size
    plt.figure(figsize=(20, 10))

    # Plot the dendrogram
    dendrogram(
        linkage_matrix,
        leaf_rotation=90.,  # rotates the x axis labels
        leaf_font_size=8.,  # font size for the x axis labels
    )

    plt.title('Community Structure Dendrogram')
    plt.xlabel('Node')
    plt.ylabel('Distance')

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    # Save the dendrogram as a PNG file
    plt.savefig('community_dendrogram.png', dpi=300, bbox_inches='tight')
    
    print("Dendrogram saved as 'community_dendrogram.png'")

    # Close the plot to free up memory
    plt.close()

In [ ]:
# Visualize the dendrogram for the wiki-vote dataset
visualise_dendogram(community_mat_wiki)

In [6]:
import time

def louvain_one_iter(nodes_connectivity_list):
    start_time = time.time()
    
    # Create a graph from the connectivity list
    G = nx.Graph()
    G.add_edges_from(nodes_connectivity_list)
    
    # Create a mapping from node IDs to contiguous indices
    node_to_index = {node: i for i, node in enumerate(G.nodes())}
    index_to_node = {i: node for node, i in node_to_index.items()}
    
    # Initialize each node to its own community
    communities = {node: i for i, node in enumerate(G.nodes())}
    
    # Calculate the initial modularity
    m = G.number_of_edges()
    Q = modularity(G, communities)
    
    total_nodes = G.number_of_nodes()
    print(f"Starting Louvain iteration for {total_nodes} nodes")
    
    # Iterate through all nodes
    for i, node in enumerate(G.nodes()):
        if i % 100 == 0:  # Print progress every 100 nodes
            elapsed_time = time.time() - start_time
            progress = (i + 1) / total_nodes
            estimated_total_time = elapsed_time / progress
            remaining_time = estimated_total_time - elapsed_time
            print(f"Processed {i+1}/{total_nodes} nodes ({progress:.2%}). "
                  f"Elapsed time: {elapsed_time:.2f}s. "
                  f"Estimated remaining time: {remaining_time:.2f}s")
        
        best_community = communities[node]
        best_gain = 0
        
        # Check the modularity gain for moving to each neighbor's community
        for neighbor in G.neighbors(node):
            gain = modularity_gain(G, node, communities[neighbor], communities, m)
            if gain > best_gain:
                best_gain = gain
                best_community = communities[neighbor]
        
        # Move the node to the best community if there's an improvement
        if best_community != communities[node]:
            communities[node] = best_community
    
    # Create the final partition array
    n = G.number_of_nodes()
    graph_partition = np.zeros(n, dtype=int)
    for node, community in communities.items():
        graph_partition[node_to_index[node]] = community
    
    total_time = time.time() - start_time
    print(f"Louvain iteration completed in {total_time:.2f} seconds")
    
    return graph_partition

def modularity(G, communities):
    m = G.number_of_edges()
    Q = 0
    for node in G.nodes():
        for neighbor in G.neighbors(node):
            if communities[node] == communities[neighbor]:
                Q += 1 - G.degree(node) * G.degree(neighbor) / (2 * m)
    return Q / (2 * m)

def modularity_gain(G, node, new_community, communities, m):
    old_community = communities[node]
    k_i = G.degree(node)
    k_i_in = sum(1 for neighbor in G.neighbors(node) if communities[neighbor] == new_community)
    k_i_out = sum(1 for neighbor in G.neighbors(node) if communities[neighbor] == old_community)
    sigma_tot = sum(G.degree(n) for n in G.nodes() if communities[n] == new_community)
    sigma_in = sum(G.degree(n) for n in G.nodes() if communities[n] == old_community)
    
    gain = (k_i_in - k_i_out) / (2 * m) - k_i * (sigma_tot - sigma_in + k_i) / (2 * m * m)
    return gain

In [52]:
# Run one iteration of Louvain algorithm for the wiki-vote dataset
graph_partition_louvain_wiki = louvain_one_iter(nodes_connectivity_list_wiki)

# Print the number of communities found
unique_communities = len(np.unique(graph_partition_louvain_wiki))
print(f"Number of communities after one Louvain iteration: {unique_communities}")

# Print the first few entries of the partition
print("First 10 entries of the partition:")
print(graph_partition_louvain_wiki[:10])

Starting Louvain iteration for 7115 nodes
Processed 1/7115 nodes (0.01%). Elapsed time: 0.36s. Estimated remaining time: 2591.50s
Processed 101/7115 nodes (1.42%). Elapsed time: 8.57s. Estimated remaining time: 595.06s
Processed 201/7115 nodes (2.83%). Elapsed time: 15.57s. Estimated remaining time: 535.58s
Processed 301/7115 nodes (4.23%). Elapsed time: 23.58s. Estimated remaining time: 533.88s
Processed 401/7115 nodes (5.64%). Elapsed time: 32.08s. Estimated remaining time: 537.16s
Processed 501/7115 nodes (7.04%). Elapsed time: 41.59s. Estimated remaining time: 549.09s
Processed 601/7115 nodes (8.45%). Elapsed time: 50.99s. Estimated remaining time: 552.65s
Processed 701/7115 nodes (9.85%). Elapsed time: 62.01s. Estimated remaining time: 567.36s
Processed 801/7115 nodes (11.26%). Elapsed time: 77.38s. Estimated remaining time: 609.98s
Processed 901/7115 nodes (12.66%). Elapsed time: 87.32s. Estimated remaining time: 602.25s
Processed 1001/7115 nodes (14.07%). Elapsed time: 99.67s. E

In [ ]:
# Create a graph from the nodes_connectivity_list_wiki
G = nx.DiGraph()
G.add_edges_from(nodes_connectivity_list_wiki)

# Create a mapping from node IDs to contiguous indices
node_to_index = {node: i for i, node in enumerate(G.nodes())}

# Get unique communities and assign colors
unique_communities = np.unique(graph_partition_louvain_wiki)
color_map = plt.colormaps['tab20']
colors = {comm: color_map(i/len(unique_communities)) for i, comm in enumerate(unique_communities)}

# Assign colors to nodes based on their community
node_colors = [colors[graph_partition_louvain_wiki[node_to_index[node]]] for node in G.nodes()]

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))
pos = nx.spring_layout(G, k=0.5, iterations=50)
nx.draw(G, pos, node_color=node_colors, node_size=20, with_labels=False, edge_color='gray', alpha=0.6, ax=ax)

# Add a title
ax.set_title("Wiki-Vote Network: Communities after one Louvain iteration", fontsize=16)

# Add a colorbar legend
sm = plt.cm.ScalarMappable(cmap=color_map, norm=plt.Normalize(vmin=0, vmax=len(unique_communities)-1))
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label('Communities', fontsize=12)
cbar.set_ticks(range(len(unique_communities)))
cbar.set_ticklabels(unique_communities)

plt.tight_layout()
plt.show()

In [8]:
def import_lastfm_asia_data(file_path):

    # Initialize an empty list to store the edges
    edges = []

    # Open and read the file
    with open(file_path, 'r') as file:
        # Skip the header line
        next(file)
        for line in file:
            # Skip comments (lines starting with '#') and empty lines
            if line.startswith('#') or line.strip() == '':
                continue
            
            # Split the line into source and target nodes
            # and convert them to integers
            source, target = map(int, line.strip().split(','))
            
            # Add the edge to the list
            edges.append([source, target])
    
    # Convert the list to a numpy array and remove duplicate edges
    nodes_connectivity_list = np.unique(np.array(edges), axis=0)
    
    return nodes_connectivity_list

In [10]:
# Load the LastFM Asia dataset
lastfm_asia_edges = import_lastfm_asia_data('Data/lastfm_asia_edges.csv')

# Print the first few entries of the dataset
print("First 10 entries of the LastFM Asia dataset:")
print(lastfm_asia_edges[:10])

# Print the shape of the dataset
print(f"\nShape of the LastFM Asia dataset: {lastfm_asia_edges.shape}")

First 10 entries of the LastFM Asia dataset:
[[   0  747]
 [   1  126]
 [   1  580]
 [   1 1222]
 [   1 2194]
 [   1 2204]
 [   1 2639]
 [   1 4257]
 [   1 5735]
 [   1 6478]]

Shape of the LastFM Asia dataset: (27806, 2)


In [ ]:
# Perform Girvan-Newman algorithm for one level on the LastFM Asia dataset
graph_partition_lastfm = Girvan_Newman_one_level(lastfm_asia_edges)

# Print the resulting graph partition
print("Graph partition for LastFM Asia dataset (one level of Girvan-Newman):")
print(graph_partition_lastfm)

In [ ]:

# Perform the full Girvan-Newman algorithm on the LastFM Asia dataset
community_mat_lastfm = Girvan_Newman(lastfm_asia_edges)

# Print the resulting community matrix
print("Community matrix for LastFM Asia dataset (full Girvan-Newman):")
print(community_mat_lastfm)

# Visualize the dendrogram for the communities obtained
visualise_dendogram(community_mat_lastfm)

In [ ]:
# Perform one iteration of the Louvain algorithm on the LastFM Asia dataset
graph_partition_louvain_lastfm = louvain_one_iter(lastfm_asia_edges)

# Print the resulting graph partition from the Louvain algorithm
print("Graph partition for LastFM Asia dataset (one iteration of Louvain):")
print(graph_partition_louvain_lastfm)